In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import tree
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import visualkeras
import tensorflow as tf
tf.data.experimental.enable_debug_mode()
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm_notebook
from sklearn.tree import export_graphviz

In [2]:
combined_df = pd.read_csv('comb_df.csv',index_col=0)
combined_df

C:\Users\gsevr\AppData\Local\Temp\ipykernel_20600\2271184525.py:1: DtypeWarning: Columns (105) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('comb_df.csv',index_col=0)


,ShotDist1,TimeoutTeam1,Substitution1,Shooter1,Rebounder1,Blocker1,Fouler1,ReboundType1,ViolationPlayer1,FreeThrowShooter1,TurnoverPlayer1,ShotDist2,TimeoutTeam2,Substitution2,Shooter2,Rebounder2,Blocker2,Fouler2,ReboundType2,ViolationPlayer2,FreeThrowShooter2,TurnoverPlayer2,ShotDist3,TimeoutTeam3,Substitution3,Shooter3,Rebounder3,Blocker3,Fouler3,ReboundType3,ViolationPlayer3,FreeThrowShooter3,TurnoverPlayer3,ShotDist4,TimeoutTeam4,Substitution4,Shooter4,Rebounder4,Blocker4,Fouler4,ReboundType4,ViolationPlayer4,FreeThrowShooter4,TurnoverPlayer4,ShotDist5,TimeoutTeam5,Substitution5,Shooter5,Rebounder5,Blocker5,Fouler5,ReboundType5,ViolationPlayer5,FreeThrowShooter5,TurnoverPlayer5,ShotDist6,TimeoutTeam6,Substitution6,Shooter6,Rebounder6,Blocker6,Fouler6,ReboundType6,ViolationPlayer6,FreeThrowShooter6,TurnoverPlayer6,ShotDist7,TimeoutTeam7,Substitution7,Shooter7,Rebounder7,Blocker7,Fouler7,ReboundType7,ViolationPlayer7,FreeThrowShooter7,TurnoverPlayer7,ShotDist8,TimeoutTeam8,Substitution8,Shooter8,Rebounder8,Blocker8,Fouler8,ReboundType8,ViolationPlayer8,FreeThrowShooter8,TurnoverPlayer8,ShotDist9,TimeoutTeam9,Substitution9,Shooter9,Rebounder9,Blocker9,Fouler9,ReboundType9,ViolationPlayer9,FreeThrowShooter9,TurnoverPlayer9,ShotDist10,TimeoutTeam10,Substitution10,Shooter10,Rebounder10,Blocker10,Fouler10,ReboundType10,ViolationPlayer10,FreeThrowShooter10,TurnoverPlayer10,class
0,NaN,NaN,NaN,NaN,NaN,NaN,fouler_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_away,NaN,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,defensive,NaN,NaN,NaN,mid,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,defensive,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_away,1
1,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,offensive,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,offensive,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_home,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,defensive,NaN,NaN,NaN,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,defensive,NaN,NaN,NaN,3pt,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,defensive,NaN,NaN,NaN,NaN,timeout_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_home,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_away,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_home

In [3]:
df = pd.DataFrame()
encoders = []

for column in combined_df.columns[:-1]:
    le = LabelEncoder()
    encoders.append(le)
    df[column] = le.fit_transform(combined_df[column])

df = pd.concat([df,combined_df.iloc[:,-1]])

C:\Users\gsevr\AppData\Local\Temp\ipykernel_20600\542520206.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_20600\542520206.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_20600\542520206.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [18]:
import pandas as pd
import time
from collections import defaultdict

events = {}

for j, event in zip(range(12,112,11), range(10, 0, -1)):
    a = combined_df.iloc[:, -j:-1][combined_df['class'] == 1]

    # count occurrences of each row
    row_counts = defaultdict(int)
    for i in range(len(a)):
        row_tuple = tuple(a.iloc[i])
        row_counts[row_tuple] += 1

    # row with the maximum count
    max_count = 0
    mc_row = None
    for row, count in row_counts.items():
        if count > max_count:
            max_count = count
            mc_row = row

    # index of the row 
    mc_idx = a.apply(lambda row: tuple(row) == mc_row, axis=1).idxmax()


    print(f"Index: {mc_idx}, Max Count: {max_count}")
    events[event] = a.iloc[mc_idx]

Index: 24578, Max Count: 2890
Index: 24585, Max Count: 1123
Index: 24592, Max Count: 431
Index: 24689, Max Count: 260
Index: 24843, Max Count: 92
Index: 24978, Max Count: 22
Index: 24788, Max Count: 10
Index: 24781, Max Count: 4
Index: 25236, Max Count: 3
Index: 25742, Max Count: 2


In [30]:
events[1].to_frame().dropna().T

,Fouler1,TurnoverPlayer2,ShotDist3,Shooter3,ShotDist4,Shooter4,Rebounder5,ReboundType5,ShotDist6,Shooter6,TimeoutTeam7,Substitution8,Substitution9,Fouler10
25742,fouler_away,to_player_away,close,shooter_home,close,shooter_away,rebounder_home,defensive,close,shooter_home,timeout_away,sub_away,sub_away,fouler_away
